<a id='top'></a>

In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


# %matplotlib nbagg 
%matplotlib notebook
# %matplotlib inline



## Third party 
import numpy as np
import os, time, zarr, sys
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl

import unslice.IO as io
from unslice.utils import *
from unslice.registration.featmatch import *
from unslice.registration.transform import *
from unslice.registration.rigid import *
from unslice.registration.gpu_transform import *
from unslice.registration.utils import *
from unslice.segmentation import *
from unslice.tracing.pyoof import OOF, apply_oof_v2
from unslice.tracing.skel import *
from unslice.flatten import *
from unslice.lightsheetcorrect import *




In [2]:
# Parameters that are constant throughout notebook
working_dir = '/mnt/share3/webster/mEhmAD_2-3'

def bdir(fname):
    return os.path.join(working_dir, fname)

# prefix to add to the beginning of each filename 
name_prefix2 = '2-abeta_4x'

# Table of contents

### Pre-processing
[1. Convert to zarr](#convert)<br>
[2. Flatten warp](#flattenwarp)<br>
[3. Lectin warp](#anchorwarp)<br>

### Transformation
[14. Rigid transformation based on manual anchor points](#rigidanchor)<br>
[15. TPS transformation based on manual anchor points (round 0)](#anchorwarp)<br>

# Convert to zarr
<a id='convert'></a>

[Return to top](#top)

In [3]:
# 2

slab_path = '/mnt/cephfs/general/juhyuk/15xdata/20220512_21_26_12_mEhmAD_2_abeta.lec.ptau/Ex_488_Em_1_destriped_stitched_tiffs_4xdownsampled'
slab_zarr_path = bdir(name_prefix2+'.zarr')
pc2_img_size = (22177,7817,5933)


## Optional parameters 
load_num_slices = 40 # should be smaller than z chunk size, default None 
resample_num_slices = 1 # number of slices to be processed in one go for resampling, default 1
resample_factor = (1,1) # original is (0.306,0.433,0.306)
chunks = (200,200,200) 
num_workers = 20 
file_names = 'img_[0-9]{4}.tiff' # default 'img_[0-9]{4}.tiff'

## crop
xrange = [3600//4,3600//4+16400//4]
yrange = [0,7816//4]
zrange = [553//4,553//4+5376//4]

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, load_num_slices=load_num_slices,
                        resample_num_slices=resample_num_slices, file_names=file_names, 
                        chunks=chunks, num_workers=num_workers, lateral_rotate_angle=lateral_rotate_angle,
                        flip=flip, crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

Processing chunk x:900-5000, y:0-1954, z:138-178


100%|██████████| 40/40 [00:38<00:00,  1.04it/s]


Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...


100%|██████████| 210/210 [00:06<00:00, 34.41it/s]

Processing chunk x:900-5000, y:0-1954, z:178-218


...

Processing chunk x:900-5000, y:0-1954, z:1458-1482


100%|██████████| 24/24 [00:30<00:00,  1.28s/it]


Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...


100%|██████████| 210/210 [00:32<00:00,  6.51it/s]


2664.5782504081726 seconds elapsed


## Flatten warp the slabs

[Return to top](#top)

<a id='flattenwarp'></a>

## Flatten TPS Warp

In [4]:
# 2
fixed_zarr_path = bdir(name_prefix2+'.zarr') 
moving_zarr_path = bdir(name_prefix2+'.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(16,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir('warping_grids/2-lec_grid_flatten.npy')




##########################

moving_pts_paths = [bdir('manual_labels/3-lec_anchor_pts_flat.npy')] # doesn't actually matter for this 
fixed_pts_paths =  [bdir('manual_labels/3-lec_anchor_pts_flat.npy')]

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
# tiff_path = warped_zarr_path[:-5]+'_tiffs'
# convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=None)

(4100, 1954, 1344)
Loading grid values...
Warping image...
Moving image size: 21.5346432 GB


100%|██████████| 1470/1470 [05:46<00:00,  4.61it/s]


Time elapsed: 9.197107 minutes


## TPS warp using lectin grid

[Return to top](#top)

<a id='anchorwarp'></a>

## Lectin warp

In [6]:
# Top slab 
fixed_zarr_path = '/mnt/share3/webster/mEhmAD_1-3_real/3-lec_flattened_anchorwarp_r1.zarr' # We have to make this the fixed zarr so that it'll warp to the correct shape 
moving_zarr_path = bdir(name_prefix2+'_flattened.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened_lectinwarp.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(16,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = bdir('warping_grids/grid_anchor_tps_r0.npy')

# Number of z slices to add to the final image 
zadd = 0


##########################

moving_pts_paths = [bdir('manual_labels/3-lec_anchor_pts_flat.npy')] # doesn't actually matter for this 
fixed_pts_paths =  [bdir('manual_labels/3-lec_anchor_pts_flat.npy')]

# anchor parameters (using the surface on the other  side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path, zadd=zadd, 
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
# Test parallel convert zarr to tiff 
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(4525, 2025, 1344)
Loading grid values...
Warping image...
Moving image size: 21.5346432 GB


100%|██████████| 1771/1771 [07:50<00:00,  4.83it/s]


Time elapsed: 10.949748 minutes
Loading z 0 - 200


100%|██████████| 200/200 [01:01<00:00,  3.23it/s]


Loading z 200 - 400


...

100%|██████████| 200/200 [01:06<00:00,  2.99it/s]


Loading z 1000 - 1200


100%|██████████| 200/200 [00:57<00:00,  3.48it/s]


Loading z 1200 - 1344


100%|██████████| 144/144 [00:41<00:00,  3.43it/s]
